# A Notebook for Extended Auxiliary Task Experiments on Combinations of UD, NER and MLM Data:

In [ ]:
# mount google drive for access to unpublished data and saving results!
from google.colab import drive
drive.mount('/content/drive')
# check if drive is present in root directory '/content'
%ls -l
%pwd

In [ ]:
# prepare/clear drive directory
%rm -r /content/BaySIDshot # should fail in initial call
%cd /content
# get a fresh clone of BaySIDshot repo with it's submodules
! git clone https://github.com/XaverKrueckl/BaySIDshot.git --recurse-submodules
# cd into BaySIDshot repo:
%cd /content/BaySIDshot/
%ls -l
%pwd

In [ ]:
# prepare MLM data to use with machamp:
# again move into BaySIDshot repo to be safe:

! bash /content/BaySIDshot/scripts/prepare_mlm_data.sh

# prepare UD data to use with machamp:
%cd /content/BaySIDshot/

! bash /content/BaySIDshot/scripts/prepare_ud_data.sh

# data for NER is split!
# Wiki train-test-dev data accessible via
# /content/BaySIDshot/BarNER/data/BarNER-final/bar-wiki-*.tsv

# Twitter train-test-dev data may not be published for public - saved in private repo
# /content/BaySIDshot/BarNER_Twitter/bar-tweet-*.tsv

# finally move into machamp to start training and predictions
%cd /content/BaySIDshot/machamp
%ls -l
%pwd

## General Checks:

In [ ]:
# make sure to have a GPU backend selected
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# install the required packages for machamp
#! cat /content/BaySIDshot/machamp/README.md | grep "requirements"

%cd /content/BaySIDshot/machamp
! pip3 install --user -r /content/BaySIDshot/machamp/requirements.txt

In [ ]:
# appends the directory /root/.local/bin to the existing PATH variable,
# allowing executables located in that directory to be run from anywhere in the shell
! export PATH=$PATH:/root/.local/bin

# check if imports for machamp are there
import tensorflow as tf
# check the version
print(tf.__version__)

# check if basic system works
! python3 /content/BaySIDshot/machamp/train.py

## Prepare evaluation data:


In [ ]:
#!bash /content/BaySIDshot/scripts/prepare_evaldata_dialects.sh
#!bash /content/BaySIDshot/scripts/prepare_evaldata_baseline.sh
!bash /content/BaySIDshot/scripts/prepare_evaldata_alllangs.sh

# if issues occur, use manually created gold set:
# /content/BaySIDshot/manual_data/alllangs_eval_data # or just dialects_eval_data, etc.

%cd /content/BaySIDshot/machamp

## MLM_NER_NLU Extended Mulit-Sequential Intermediate setting:

In [ ]:
# inspect configs to be used
! cat /content/BaySIDshot/configs/mlm_x.json
! cat /content/BaySIDshot/configs/ner_x.json
! cat /content/BaySIDshot/configs/nlu_x.json

# inspect params to be used
! cat /content/BaySIDshot/configs/params_mdeberta.json

In [ ]:
# train experiment continuous mlm pretraining of mDeBERTa
# make sure be in ../machamp or use direct path as below
# set name of experiment / logs directory! --name mDeBERTa_exp4_mlm_ner_nlu_SEED
# with respective random seed that should be used --seed 1234 e.g.

! python3 /content/BaySIDshot/machamp/train.py --dataset_configs /content/BaySIDshot/configs/mlm_x.json /content/BaySIDshot/configs/ner_x.json /content/BaySIDshot/configs/nlu_x.json --parameters /content/BaySIDshot/configs/params_mdeberta.json --sequential --device 0 --name mDeBERTa_exp4_mlm_ner_nlu_1234 --seed 1234

In [ ]:
# save logs dir with model and metrics to drive - change name of experiment accordingly!

! cp -R /content/BaySIDshot/machamp/logs/mDeBERTa_exp4_mlm_ner_nlu_1234* /content/drive/MyDrive/Masterarbeit

In [ ]:
# SET and get path(s) to the final model
# last line with text in model_path has model path to nlu model
# (if nlu config was set as last one when sequential/intermediate training)
! ls -d /content/drive/MyDrive/Masterarbeit/mDeBERTa_exp4_mlm_ner_nlu_1234*/*/model_* > model_path.txt

import os

with open('model_path.txt', 'r') as file:
  lines = file.readlines()
  # get the path to the final model (maily if sequential experiment)
  model_line = [line.strip() for line in lines if line.strip()][-1]
  model_line = model_line.strip()
  print("Evaluating Model: ", model_line)
  if '/' in model_line:
    # just to show the parts of the path:
    parts = model_line.split('/')
    # get all the necessary path parts:
    model = model_line.split('/')[7]
    time = model_line.split('/')[6]
    experiment_name = model_line.split('/')[5]
    if experiment_name.split("."):
      experiment_name_cleaned = experiment_name.split(".")[0]
    save_dir = model_line.split('/')[:5]
    base_save_dir = '/'.join(save_dir)
  else:
    raise ValueError("No valid model path")


#eval_dir = "/content/BaySIDshot/manual_data/alllangs_eval_data"
eval_dir = "/content/BaySIDshot/alllangs_eval_data"
# other evaluation sets can be used here


with open('script.sh', 'w') as file:
    # create predictions folder in directory where model is saved:
    file.write(f"! mkdir -p {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # prepare prediction files order
    file_list = []
    for filename in os.listdir(eval_dir):
        if filename.endswith('test.conll'): # do not use when also using valid files is desired
          # path to goldfile:
          goldfile = eval_dir + "/" + filename
          # path to outfile
          outfile = base_save_dir + "/" + experiment_name + "/" + time + "/predictions_" + experiment_name_cleaned + "/" + filename + ".out"
          # append goldfile outfile "pairs" to filelist for prediction command
          file_list.append(str(goldfile))
          file_list.append(str(outfile))
    file_list_string = ' '.join(file_list)
    # prediction call for all files in test dir
    # also adds the specific dataset for which prediction should be done - necessary in multitask setting, else possible to also append
    file.write(f"! python3 predict.py {model_line} {file_list_string} --device 0 --dataset NLU\n")
    # dir eval call to get all metrics and save output as json file:
    file.write(f"! python3 /content/BaySIDshot/scripts/dir_eval_out.py {eval_dir} {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # copy json output also to predictions dir:
    file.write(f"! mkdir -p {base_save_dir}/results\n")
    file.write(f"! cp /content/BaySIDshot/results/* {base_save_dir}/results\n")

print("Prediction Script successfully generated!")

# runs predictions:
! bash script.sh
# and removes scripts:
! rm script.sh
! rm model_path.txt

## MLMxNER_NLU Extended Sequential Intermediate Multitask setting:

In [ ]:
# inspect configs to be used
! cat /content/BaySIDshot/configs/mlm_ner_x.json
! cat /content/BaySIDshot/configs/nlu_x.json

# inspect params to be used
! cat /content/BaySIDshot/configs/params_mdeberta.json

In [ ]:
# train experiment continuous mlm pretraining of mDeBERTa
# make sure be in ../machamp or use direct path as below
# set name of experiment / logs directory! --name mDeBERTa_exp4_mlmner_nlu_SEED
# with respective random seed that should be used --seed 1234 e.g.

! python3 /content/BaySIDshot/machamp/train.py --dataset_configs /content/BaySIDshot/configs/mlm_ner_x.json /content/BaySIDshot/configs/nlu_x.json --parameters /content/BaySIDshot/configs/params_mdeberta.json --sequential --device 0 --name mDeBERTa_exp4_mlmner_nlu_1234 --seed 1234

In [ ]:
# save logs dir with model and metrics to drive - change name of experiment accordingly!

! cp -R /content/BaySIDshot/machamp/logs/mDeBERTa_exp4_mlmner_nlu_1234* /content/drive/MyDrive/Masterarbeit

In [ ]:
# SET and get path(s) to the final model
# last line with text in model_path has model path to nlu model
# (if nlu config was set as last one when sequential/intermediate training)
! ls -d /content/drive/MyDrive/Masterarbeit/mDeBERTa_exp4_mlmner_nlu_1234*/*/model_* > model_path.txt

import os

with open('model_path.txt', 'r') as file:
  lines = file.readlines()
  # get the path to the final model (maily if sequential experiment)
  model_line = [line.strip() for line in lines if line.strip()][-1]
  model_line = model_line.strip()
  print("Evaluating Model: ", model_line)
  if '/' in model_line:
    # just to show the parts of the path:
    parts = model_line.split('/')
    # get all the necessary path parts:
    model = model_line.split('/')[7]
    time = model_line.split('/')[6]
    experiment_name = model_line.split('/')[5]
    if experiment_name.split("."):
      experiment_name_cleaned = experiment_name.split(".")[0]
    save_dir = model_line.split('/')[:5]
    base_save_dir = '/'.join(save_dir)
  else:
    raise ValueError("No valid model path")


#eval_dir = "/content/BaySIDshot/manual_data/alllangs_eval_data"
eval_dir = "/content/BaySIDshot/alllangs_eval_data"

with open('script.sh', 'w') as file:
    # create predictions folder in directory where model is saved:
    file.write(f"! mkdir -p {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # prepare prediction files order
    file_list = []
    for filename in os.listdir(eval_dir):
        if filename.endswith('test.conll'): # do not use when also using valid files is desired
          # path to goldfile:
          goldfile = eval_dir + "/" + filename
          # path to outfile
          outfile = base_save_dir + "/" + experiment_name + "/" + time + "/predictions_" + experiment_name_cleaned + "/" + filename + ".out"
          # append goldfile outfile "pairs" to filelist for prediction command
          file_list.append(str(goldfile))
          file_list.append(str(outfile))
    file_list_string = ' '.join(file_list)
    # prediction call for all files in test dir
    # also adds the specific dataset for which prediction should be done - necessary in multitask setting, else possible to also append
    file.write(f"! python3 predict.py {model_line} {file_list_string} --device 0 --dataset NLU\n")
    # dir eval call to get all metrics and save output as json file:
    file.write(f"! python3 /content/BaySIDshot/scripts/dir_eval_out.py {eval_dir} {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # copy json output also to predictions dir:
    file.write(f"! mkdir -p {base_save_dir}/results\n")
    file.write(f"! cp /content/BaySIDshot/results/* {base_save_dir}/results\n")

print("Prediction Script successfully generated!")

# runs predictions:
! bash script.sh
# and removes scripts:
! rm script.sh
! rm model_path.txt

## MLMxNERxNLU Extended Multitask setting:

In [ ]:
# inspect configs to be used
! cat /content/BaySIDshot/configs/mlm_ner_nlu_x.json

# inspect params to be used
! cat /content/BaySIDshot/configs/params_mdeberta.json

In [ ]:
# train experiment continuous mlm pretraining of mDeBERTa
# make sure be in ../machamp or use direct path as below
# set name of experiment / logs directory! --name mDeBERTa_exp4_mlmnernlu_SEED
# with respective random seed that should be used --seed 1234 e.g.

! python3 /content/BaySIDshot/machamp/train.py --dataset_configs /content/BaySIDshot/configs/mlm_ner_nlu_x.json --parameters /content/BaySIDshot/configs/params_mdeberta.json  --device 0 --name mDeBERTa_exp4_mlmnernlu_1234 --seed 1234

In [ ]:
# save logs dir with model and metrics to drive - change name of experiment accordingly!

! cp -R /content/BaySIDshot/machamp/logs/mDeBERTa_exp4_mlmnernlu_1234* /content/drive/MyDrive/Masterarbeit

In [ ]:
# SET and get path(s) to the final model
# last line with text in model_path has model path to nlu model
# (if nlu config was set as last one when sequential/intermediate training)
! ls -d /content/drive/MyDrive/Masterarbeit/mDeBERTa_exp4_mlmnernlu_1234*/*/model_* > model_path.txt

import os

with open('model_path.txt', 'r') as file:
  lines = file.readlines()
  # get the path to the final model (maily if sequential experiment)
  model_line = [line.strip() for line in lines if line.strip()][-1]
  model_line = model_line.strip()
  print("Evaluating Model: ", model_line)
  if '/' in model_line:
    # just to show the parts of the path:
    parts = model_line.split('/')
    # get all the necessary path parts:
    model = model_line.split('/')[7]
    time = model_line.split('/')[6]
    experiment_name = model_line.split('/')[5]
    if experiment_name.split("."):
      experiment_name_cleaned = experiment_name.split(".")[0]
    save_dir = model_line.split('/')[:5]
    base_save_dir = '/'.join(save_dir)
  else:
    raise ValueError("No valid model path")


#eval_dir = "/content/BaySIDshot/manual_data/alllangs_eval_data"
eval_dir = "/content/BaySIDshot/alllangs_eval_data"

with open('script.sh', 'w') as file:
    # create predictions folder in directory where model is saved:
    file.write(f"! mkdir -p {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # prepare prediction files order
    file_list = []
    for filename in os.listdir(eval_dir):
        if filename.endswith('test.conll'): # do not use when also using valid files is desired
          # path to goldfile:
          goldfile = eval_dir + "/" + filename
          # path to outfile
          outfile = base_save_dir + "/" + experiment_name + "/" + time + "/predictions_" + experiment_name_cleaned + "/" + filename + ".out"
          # append goldfile outfile "pairs" to filelist for prediction command
          file_list.append(str(goldfile))
          file_list.append(str(outfile))
    file_list_string = ' '.join(file_list)
    # prediction call for all files in test dir
    # also adds the specific dataset for which prediction should be done - necessary in multitask setting, else possible to also append
    file.write(f"! python3 predict.py {model_line} {file_list_string} --device 0 --dataset NLU\n")
    # dir eval call to get all metrics and save output as json file:
    file.write(f"! python3 /content/BaySIDshot/scripts/dir_eval_out.py {eval_dir} {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # copy json output also to predictions dir:
    file.write(f"! mkdir -p {base_save_dir}/results\n")
    file.write(f"! cp /content/BaySIDshot/results/* {base_save_dir}/results\n")

print("Prediction Script successfully generated!")

# runs predictions:
! bash script.sh
# and removes scripts:
! rm script.sh
! rm model_path.txt

## MLMxUD_NLU Sequential Intermediate Multitask setting:

In [ ]:
# inspect configs to be used
! cat /content/BaySIDshot/configs/mlm_ud_x.json
! cat /content/BaySIDshot/configs/nlu_x.json

# inspect params to be used
! cat /content/BaySIDshot/configs/params_mdeberta.json

In [ ]:
# train experiment continuous mlm pretraining of mDeBERTa
# make sure be in ../machamp or use direct path as below
# set name of experiment / logs directory! --name mDeBERTa_exp4_mlmud_nlu_SEED
# with respective random seed that should be used --seed 1234 e.g.

! python3 /content/BaySIDshot/machamp/train.py --dataset_configs /content/BaySIDshot/configs/mlm_ud_x.json /content/BaySIDshot/configs/nlu_x.json --parameters /content/BaySIDshot/configs/params_mdeberta.json --sequential --device 0 --name mDeBERTa_exp4_mlmud_nlu_1234 --seed 1234

In [ ]:
# save logs dir with model and metrics to drive - change name of experiment accordingly!

! cp -R /content/BaySIDshot/machamp/logs/mDeBERTa_exp4_mlmud_nlu_1234* /content/drive/MyDrive/Masterarbeit

In [ ]:
# SET and get path(s) to the final model
# last line with text in model_path has model path to nlu model
# (if nlu config was set as last one when sequential/intermediate training)
! ls -d /content/drive/MyDrive/Masterarbeit/mDeBERTa_exp4_mlmud_nlu_1234*/*/model_* > model_path.txt

import os

with open('model_path.txt', 'r') as file:
  lines = file.readlines()
  # get the path to the final model (maily if sequential experiment)
  model_line = [line.strip() for line in lines if line.strip()][-1]
  model_line = model_line.strip()
  print("Evaluating Model: ", model_line)
  if '/' in model_line:
    # just to show the parts of the path:
    parts = model_line.split('/')
    # get all the necessary path parts:
    model = model_line.split('/')[7]
    time = model_line.split('/')[6]
    experiment_name = model_line.split('/')[5]
    if experiment_name.split("."):
      experiment_name_cleaned = experiment_name.split(".")[0]
    save_dir = model_line.split('/')[:5]
    base_save_dir = '/'.join(save_dir)
  else:
    raise ValueError("No valid model path")


#eval_dir = "/content/BaySIDshot/manual_data/alllangs_eval_data"
eval_dir = "/content/BaySIDshot/alllangs_eval_data"


with open('script.sh', 'w') as file:
    # create predictions folder in directory where model is saved:
    file.write(f"! mkdir -p {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # prepare prediction files order
    file_list = []
    for filename in os.listdir(eval_dir):
        if filename.endswith('test.conll'): # do not use when also using valid files is desired
          # path to goldfile:
          goldfile = eval_dir + "/" + filename
          # path to outfile
          outfile = base_save_dir + "/" + experiment_name + "/" + time + "/predictions_" + experiment_name_cleaned + "/" + filename + ".out"
          # append goldfile outfile "pairs" to filelist for prediction command
          file_list.append(str(goldfile))
          file_list.append(str(outfile))
    file_list_string = ' '.join(file_list)
    # prediction call for all files in test dir
    # also adds the specific dataset for which prediction should be done - necessary in multitask setting, else possible to also append
    file.write(f"! python3 predict.py {model_line} {file_list_string} --device 0 --dataset NLU\n")
    # dir eval call to get all metrics and save output as json file:
    file.write(f"! python3 /content/BaySIDshot/scripts/dir_eval_out.py {eval_dir} {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # copy json output also to predictions dir:
    file.write(f"! mkdir -p {base_save_dir}/results\n")
    file.write(f"! cp /content/BaySIDshot/results/* {base_save_dir}/results\n")

print("Prediction Script successfully generated!")

# runs predictions:
! bash script.sh
# and removes scripts:
! rm script.sh
! rm model_path.txt

## Extended MLMxUDxNERxNLU Multitask setting:

In [ ]:
# inspect configs to be used
! cat /content/BaySIDshot/configs/mulm_ud_ner_nlu_x.json

# inspect params to be used
! cat /content/BaySIDshot/configs/params_mdeberta.json

In [ ]:
# train experiment continuous mlm pretraining of mDeBERTa
# make sure be in ../machamp or use direct path as below
# set name of experiment / logs directory! --name mDeBERTa_exp4_mlmudnernlu_SEED
# with respective random seed that should be used --seed 1234 e.g.

! python3 /content/BaySIDshot/machamp/train.py --dataset_configs /content/BaySIDshot/configs/mulm_ud_ner_nlu_x.json --parameters /content/BaySIDshot/configs/params_mdeberta.json  --device 0 --name mDeBERTa_exp4_mlmudnernlu_1234 --seed 1234

In [ ]:
# save logs dir with model and metrics to drive - change name of experiment accordingly!

! cp -R /content/BaySIDshot/machamp/logs/mDeBERTa_exp4_mlmudnernlu_1234* /content/drive/MyDrive/Masterarbeit

In [ ]:
# SET and get path(s) to the final model
# last line with text in model_path has model path to nlu model
# (if nlu config was set as last one when sequential/intermediate training)
! ls -d /content/drive/MyDrive/Masterarbeit/mDeBERTa_exp4_mlmudnernlu_1234*/*/model_* > model_path.txt

import os

with open('model_path.txt', 'r') as file:
  lines = file.readlines()
  # get the path to the final model (maily if sequential experiment)
  model_line = [line.strip() for line in lines if line.strip()][-1]
  model_line = model_line.strip()
  print("Evaluating Model: ", model_line)
  if '/' in model_line:
    # just to show the parts of the path:
    parts = model_line.split('/')
    # get all the necessary path parts:
    model = model_line.split('/')[7]
    time = model_line.split('/')[6]
    experiment_name = model_line.split('/')[5]
    if experiment_name.split("."):
      experiment_name_cleaned = experiment_name.split(".")[0]
    save_dir = model_line.split('/')[:5]
    base_save_dir = '/'.join(save_dir)
  else:
    raise ValueError("No valid model path")


#eval_dir = "/content/BaySIDshot/manual_data/alllangs_eval_data"
eval_dir = "/content/BaySIDshot/alllangs_eval_data"

with open('script.sh', 'w') as file:
    # create predictions folder in directory where model is saved:
    file.write(f"! mkdir -p {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # prepare prediction files order
    file_list = []
    for filename in os.listdir(eval_dir):
        if filename.endswith('test.conll'): # do not use when also using valid files is desired
          # path to goldfile:
          goldfile = eval_dir + "/" + filename
          # path to outfile
          outfile = base_save_dir + "/" + experiment_name + "/" + time + "/predictions_" + experiment_name_cleaned + "/" + filename + ".out"
          # append goldfile outfile "pairs" to filelist for prediction command
          file_list.append(str(goldfile))
          file_list.append(str(outfile))
    file_list_string = ' '.join(file_list)
    # prediction call for all files in test dir
    # also adds the specific dataset for which prediction should be done - necessary in multitask setting, else possible to also append
    file.write(f"! python3 predict.py {model_line} {file_list_string} --device 0 --dataset NLU\n")
    # dir eval call to get all metrics and save output as json file:
    file.write(f"! python3 /content/BaySIDshot/scripts/dir_eval_out.py {eval_dir} {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # copy json output also to predictions dir:
    file.write(f"! mkdir -p {base_save_dir}/results\n")
    file.write(f"! cp /content/BaySIDshot/results/* {base_save_dir}/results\n")

print("Prediction Script successfully generated!")

# runs predictions:
! bash script.sh
# and removes scripts:
! rm script.sh
! rm model_path.txt

## UD_NER_NLU Extended Mulit-Sequential Intermediate setting:

In [ ]:
# inspect configs to be used
! cat /content/BaySIDshot/configs/ud_x.json
! cat /content/BaySIDshot/configs/ner_x.json
! cat /content/BaySIDshot/configs/nlu_x.json

# inspect params to be used
! cat /content/BaySIDshot/configs/params_mdeberta.json

In [ ]:
# train experiment continuous mlm pretraining of mDeBERTa
# make sure be in ../machamp or use direct path as below
# set name of experiment / logs directory! --name mDeBERTa_exp4_ud_ner_nlu_SEED
# with respective random seed that should be used --seed 1234 e.g.

! python3 /content/BaySIDshot/machamp/train.py --dataset_configs /content/BaySIDshot/configs/ud_x.json /content/BaySIDshot/configs/ner_x.json /content/BaySIDshot/configs/nlu_x.json --parameters /content/BaySIDshot/configs/params_mdeberta.json --sequential --device 0 --name mDeBERTa_exp4_ud_ner_nlu_1234 --seed 1234

In [ ]:
# save logs dir with model and metrics to drive - change name of experiment accordingly!

! cp -R /content/BaySIDshot/machamp/logs/mDeBERTa_exp4_ud_ner_nlu_1234* /content/drive/MyDrive/Masterarbeit

In [ ]:
# SET and get path(s) to the final model
# last line with text in model_path has model path to nlu model
# (if nlu config was set as last one when sequential/intermediate training)
! ls -d /content/drive/MyDrive/Masterarbeit/mDeBERTa_exp4_ud_ner_nlu_1234*/*/model_* > model_path.txt

import os

with open('model_path.txt', 'r') as file:
  lines = file.readlines()
  # get the path to the final model (maily if sequential experiment)
  model_line = [line.strip() for line in lines if line.strip()][-1]
  model_line = model_line.strip()
  print("Evaluating Model: ", model_line)
  if '/' in model_line:
    # just to show the parts of the path:
    parts = model_line.split('/')
    # get all the necessary path parts:
    model = model_line.split('/')[7]
    time = model_line.split('/')[6]
    experiment_name = model_line.split('/')[5]
    if experiment_name.split("."):
      experiment_name_cleaned = experiment_name.split(".")[0]
    save_dir = model_line.split('/')[:5]
    base_save_dir = '/'.join(save_dir)
  else:
    raise ValueError("No valid model path")


#eval_dir = "/content/BaySIDshot/manual_data/alllangs_eval_data"
eval_dir = "/content/BaySIDshot/alllangs_eval_data"

with open('script.sh', 'w') as file:
    # create predictions folder in directory where model is saved:
    file.write(f"! mkdir -p {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # prepare prediction files order
    file_list = []
    for filename in os.listdir(eval_dir):
        if filename.endswith('test.conll'): # do not use when also using valid files is desired
          # path to goldfile:
          goldfile = eval_dir + "/" + filename
          # path to outfile
          outfile = base_save_dir + "/" + experiment_name + "/" + time + "/predictions_" + experiment_name_cleaned + "/" + filename + ".out"
          # append goldfile outfile "pairs" to filelist for prediction command
          file_list.append(str(goldfile))
          file_list.append(str(outfile))
    file_list_string = ' '.join(file_list)
    # prediction call for all files in test dir
    # also adds the specific dataset for which prediction should be done - necessary in multitask setting, else possible to also append
    file.write(f"! python3 predict.py {model_line} {file_list_string} --device 0 --dataset NLU\n")
    # dir eval call to get all metrics and save output as json file:
    file.write(f"! python3 /content/BaySIDshot/scripts/dir_eval_out.py {eval_dir} {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # copy json output also to predictions dir:
    file.write(f"! mkdir -p {base_save_dir}/results\n")
    file.write(f"! cp /content/BaySIDshot/results/* {base_save_dir}/results\n")

print("Prediction Script successfully generated!")

# runs predictions:
! bash script.sh
# and removes scripts:
! rm script.sh
! rm model_path.txt

## UDxNER_NLU Extended Sequential Intermediate Multitask setting:

In [ ]:
# inspect configs to be used
! cat /content/BaySIDshot/configs/ud_ner_x.json
! cat /content/BaySIDshot/configs/nlu_x.json

# inspect params to be used
! cat /content/BaySIDshot/configs/params_mdeberta.json

In [ ]:
# train experiment continuous mlm pretraining of mDeBERTa
# make sure be in ../machamp or use direct path as below
# set name of experiment / logs directory! --name mDeBERTa_exp4_udner_nlu_SEED
# with respective random seed that should be used --seed 1234 e.g.

! python3 /content/BaySIDshot/machamp/train.py --dataset_configs /content/BaySIDshot/configs/ud_ner_x.json /content/BaySIDshot/configs/nlu_x.json --parameters /content/BaySIDshot/configs/params_mdeberta.json --sequential --device 0 --name mDeBERTa_exp4_udner_nlu_1234 --seed 1234

In [ ]:
# save logs dir with model and metrics to drive - change name of experiment accordingly!

! cp -R /content/BaySIDshot/machamp/logs/mDeBERTa_exp4_udner_nlu_1234* /content/drive/MyDrive/Masterarbeit

In [ ]:
# SET and get path(s) to the final model
# last line with text in model_path has model path to nlu model
# (if nlu config was set as last one when sequential/intermediate training)
! ls -d /content/drive/MyDrive/Masterarbeit/mDeBERTa_exp4_udner_nlu_1234*/*/model_* > model_path.txt

import os

with open('model_path.txt', 'r') as file:
  lines = file.readlines()
  # get the path to the final model (maily if sequential experiment)
  model_line = [line.strip() for line in lines if line.strip()][-1]
  model_line = model_line.strip()
  print("Evaluating Model: ", model_line)
  if '/' in model_line:
    # just to show the parts of the path:
    parts = model_line.split('/')
    # get all the necessary path parts:
    model = model_line.split('/')[7]
    time = model_line.split('/')[6]
    experiment_name = model_line.split('/')[5]
    if experiment_name.split("."):
      experiment_name_cleaned = experiment_name.split(".")[0]
    save_dir = model_line.split('/')[:5]
    base_save_dir = '/'.join(save_dir)
  else:
    raise ValueError("No valid model path")


#eval_dir = "/content/BaySIDshot/manual_data/alllangs_eval_data"
eval_dir = "/content/BaySIDshot/alllangs_eval_data"

with open('script.sh', 'w') as file:
    # create predictions folder in directory where model is saved:
    file.write(f"! mkdir -p {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # prepare prediction files order
    file_list = []
    for filename in os.listdir(eval_dir):
        if filename.endswith('test.conll'): # do not use when also using valid files is desired
          # path to goldfile:
          goldfile = eval_dir + "/" + filename
          # path to outfile
          outfile = base_save_dir + "/" + experiment_name + "/" + time + "/predictions_" + experiment_name_cleaned + "/" + filename + ".out"
          # append goldfile outfile "pairs" to filelist for prediction command
          file_list.append(str(goldfile))
          file_list.append(str(outfile))
    file_list_string = ' '.join(file_list)
    # prediction call for all files in test dir
    # also adds the specific dataset for which prediction should be done - necessary in multitask setting, else possible to also append
    file.write(f"! python3 predict.py {model_line} {file_list_string} --device 0 --dataset NLU\n")
    # dir eval call to get all metrics and save output as json file:
    file.write(f"! python3 /content/BaySIDshot/scripts/dir_eval_out.py {eval_dir} {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # copy json output also to predictions dir:
    file.write(f"! mkdir -p {base_save_dir}/results\n")
    file.write(f"! cp /content/BaySIDshot/results/* {base_save_dir}/results\n")

print("Prediction Script successfully generated!")

# runs predictions:
! bash script.sh
# and removes scripts:
! rm script.sh
! rm model_path.txt

## UDxNERxNLU Extended Multitask Setting:

In [ ]:
# inspect configs to be used
! cat /content/BaySIDshot/configs/ud_ner_nlu_x.json

# inspect params to be used
! cat /content/BaySIDshot/configs/params_mdeberta.json

In [ ]:
# train experiment continuous mlm pretraining of mDeBERTa
# make sure be in ../machamp or use direct path as below
# set name of experiment / logs directory! --name mDeBERTa_exp4_udnernlu_SEED
# with respective random seed that should be used --seed 1234 e.g.

! python3 /content/BaySIDshot/machamp/train.py --dataset_configs /content/BaySIDshot/configs/ud_ner_nlu_x.json --parameters /content/BaySIDshot/configs/params_mdeberta.json  --device 0 --name mDeBERTa_exp4_udnernlu_1234 --seed 1234

In [ ]:
# save logs dir with model and metrics to drive - change name of experiment accordingly!

! cp -R /content/BaySIDshot/machamp/logs/mDeBERTa_exp4_udnernlu_1234* /content/drive/MyDrive/Masterarbeit

In [ ]:
# SET and get path(s) to the final model
# last line with text in model_path has model path to nlu model
# (if nlu config was set as last one when sequential/intermediate training)
! ls -d /content/drive/MyDrive/Masterarbeit/mDeBERTa_exp4_udnernlu_1234*/*/model_* > model_path.txt

import os

with open('model_path.txt', 'r') as file:
  lines = file.readlines()
  # get the path to the final model (maily if sequential experiment)
  model_line = [line.strip() for line in lines if line.strip()][-1]
  model_line = model_line.strip()
  print("Evaluating Model: ", model_line)
  if '/' in model_line:
    # just to show the parts of the path:
    parts = model_line.split('/')
    # get all the necessary path parts:
    model = model_line.split('/')[7]
    time = model_line.split('/')[6]
    experiment_name = model_line.split('/')[5]
    if experiment_name.split("."):
      experiment_name_cleaned = experiment_name.split(".")[0]
    save_dir = model_line.split('/')[:5]
    base_save_dir = '/'.join(save_dir)
  else:
    raise ValueError("No valid model path")


#eval_dir = "/content/BaySIDshot/manual_data/alllangs_eval_data"
eval_dir = "/content/BaySIDshot/alllangs_eval_data"

with open('script.sh', 'w') as file:
    # create predictions folder in directory where model is saved:
    file.write(f"! mkdir -p {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # prepare prediction files order
    file_list = []
    for filename in os.listdir(eval_dir):
        if filename.endswith('test.conll'): # do not use when also using valid files is desired
          # path to goldfile:
          goldfile = eval_dir + "/" + filename
          # path to outfile
          outfile = base_save_dir + "/" + experiment_name + "/" + time + "/predictions_" + experiment_name_cleaned + "/" + filename + ".out"
          # append goldfile outfile "pairs" to filelist for prediction command
          file_list.append(str(goldfile))
          file_list.append(str(outfile))
    file_list_string = ' '.join(file_list)
    # prediction call for all files in test dir
    # also adds the specific dataset for which prediction should be done - necessary in multitask setting, else possible to also append
    file.write(f"! python3 predict.py {model_line} {file_list_string} --device 0 --dataset NLU\n")
    # dir eval call to get all metrics and save output as json file:
    file.write(f"! python3 /content/BaySIDshot/scripts/dir_eval_out.py {eval_dir} {base_save_dir}/{experiment_name}/{time}/predictions_{experiment_name_cleaned}\n")
    # copy json output also to predictions dir:
    file.write(f"! mkdir -p {base_save_dir}/results\n")
    file.write(f"! cp /content/BaySIDshot/results/* {base_save_dir}/results\n")

print("Prediction Script successfully generated!")

# runs predictions:
! bash script.sh
# and removes scripts:
! rm script.sh
! rm model_path.txt

### Reference for _MaChAmp_ as well as UD, NER and MLM Data:

```
@inproceedings{van-der-goot-etal-2021-massive,
    title = "Massive Choice, Ample Tasks ({M}a{C}h{A}mp): A Toolkit for Multi-task Learning in {NLP}",
    author = {van der Goot, Rob  and
      {\"U}st{\"u}n, Ahmet  and
      Ramponi, Alan  and
      Sharaf, Ibrahim  and
      Plank, Barbara},
    booktitle = "Proceedings of the 16th Conference of the European Chapter of the Association for Computational Linguistics: System Demonstrations",
    month = apr,
    year = "2021",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.eacl-demos.22",
    doi = "10.18653/v1/2021.eacl-demos.22",
    pages = "176--197",
    abstract = "Transfer learning, particularly approaches that combine multi-task learning with pre-trained contextualized embeddings and fine-tuning, have advanced the field of Natural Language Processing tremendously in recent years. In this paper we present MaChAmp, a toolkit for easy fine-tuning of contextualized embeddings in multi-task settings. The benefits of MaChAmp are its flexible configuration options, and the support of a variety of natural language processing tasks in a uniform toolkit, from text classification and sequence labeling to dependency parsing, masked language modeling, and text generation.",
}
@inproceedings{blaschke-etal-2024-maibaam,
    title = "{M}ai{B}aam: A Multi-Dialectal {B}avarian {U}niversal {D}ependency Treebank",
    author = {Blaschke, Verena  and
      Kova{\v{c}}i{\'c}, Barbara  and
      Peng, Siyao  and
      Sch{\"u}tze, Hinrich  and
      Plank, Barbara},
    editor = "Calzolari, Nicoletta  and
      Kan, Min-Yen  and
      Hoste, Veronique  and
      Lenci, Alessandro  and
      Sakti, Sakriani  and
      Xue, Nianwen",
    booktitle = "Proceedings of the 2024 Joint International Conference on Computational Linguistics, Language Resources and Evaluation (LREC-COLING 2024)",
    month = may,
    year = "2024",
    address = "Torino, Italia",
    publisher = "ELRA and ICCL",
    url = "https://aclanthology.org/2024.lrec-main.953",
    pages = "10921--10938",
    abstract = "Despite the success of the Universal Dependencies (UD) project exemplified by its impressive language breadth, there is still a lack in {`}within-language breadth{'}: most treebanks focus on standard languages. Even for German, the language with the most annotations in UD, so far no treebank exists for one of its language varieties spoken by over 10M people: Bavarian. To contribute to closing this gap, we present the first multi-dialect Bavarian treebank (MaiBaam) manually annotated with part-of-speech and syntactic dependency information in UD, covering multiple text genres (wiki, fiction, grammar examples, social, non-fiction). We highlight the morphosyntactic differences between the closely-related Bavarian and German and showcase the rich variability of speakers{'} orthographies. Our corpus includes 15k tokens, covering dialects from all Bavarian-speaking areas spanning three countries. We provide baseline parsing and POS tagging results, which are lower than results obtained on German and vary substantially between different graph-based parsers. To support further research on Bavarian syntax, we make our dataset, language-specific guidelines and code publicly available.",
}
@inproceedings{peng-etal-2024-sebastian,
    title = "Sebastian, Basti, Wastl?! Recognizing Named Entities in {B}avarian Dialectal Data",
    author = "Peng, Siyao  and
      Sun, Zihang  and
      Shan, Huangyan  and
      Kolm, Marie  and
      Blaschke, Verena  and
      Artemova, Ekaterina  and
      Plank, Barbara",
    editor = "Calzolari, Nicoletta  and
      Kan, Min-Yen  and
      Hoste, Veronique  and
      Lenci, Alessandro  and
      Sakti, Sakriani  and
      Xue, Nianwen",
    booktitle = "Proceedings of the 2024 Joint International Conference on Computational Linguistics, Language Resources and Evaluation (LREC-COLING 2024)",
    month = may,
    year = "2024",
    address = "Torino, Italia",
    publisher = "ELRA and ICCL",
    url = "https://aclanthology.org/2024.lrec-main.1262",
    pages = "14478--14493",
    abstract = "Named Entity Recognition (NER) is a fundamental task to extract key information from texts, but annotated resources are scarce for dialects. This paper introduces the first dialectal NER dataset for German, BarNER, with 161K tokens annotated on Bavarian Wikipedia articles (bar-wiki) and tweets (bar-tweet), using a schema adapted from German CoNLL 2006 and GermEval. The Bavarian dialect differs from standard German in lexical distribution, syntactic construction, and entity information. We conduct in-domain, cross-domain, sequential, and joint experiments on two Bavarian and three German corpora and present the first comprehensive NER results on Bavarian. Incorporating knowledge from the larger German NER (sub-)datasets notably improves on bar-wiki and moderately on bar-tweet. Inversely, training first on Bavarian contributes slightly to the seminal German CoNLL 2006 corpus. Moreover, with gold dialect labels on Bavarian tweets, we assess multi-task learning between five NER and two Bavarian-German dialect identification tasks and achieve NER SOTA on bar-wiki. We substantiate the necessity of our low-resource BarNER corpus and the importance of diversity in dialects, genres, and topics in enhancing model performance.",
}
@inproceedings{artemova-plank-2023-low,
    title = "Low-resource Bilingual Dialect Lexicon Induction with Large Language Models",
    author = "Artemova, Ekaterina  and Plank, Barbara",
    booktitle = "Proceedings of the 24th Nordic Conference on Computational Linguistics (NoDaLiDa 2023) (NoDaLiDa)",
    year = "2023",
}

```